In [1]:
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.messages import SystemMessage, HumanMessage 

c:\Users\xgear\miniconda3\envs\intern\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

# LLM cho chat
llm = ChatOllama(
    model="gemma3:1b",
    base_url="http://localhost:11434"
)

# Embedding model
embedding = OllamaEmbeddings(
    model="embeddinggemma:300m",
    base_url="http://localhost:11434"
)

C:\Users\xgear\AppData\Local\Temp\ipykernel_25092\3321198127.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(
C:\Users\xgear\AppData\Local\Temp\ipykernel_25092\3321198127.py:11: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(


In [3]:
def run_reflection_loop():
    task_prompt = """
    Your task is to create a Python function named `calculate_factorial`. 
    This function should do the following: 
    1.  Accept a single integer `n` as input.
    2.  Calculate its factorial (n!). 
    3.  Include a clear docstring explaining what the function does. 
    4.  Handle edge cases: The factorial of 0 is 1. 
    5.  Handle invalid input: Raise a ValueError if the input is a 
    negative number.
    """ 
    max_iterations = 3 
    current_code = "" 
    message_history = [HumanMessage(content=task_prompt)] 
    for i in range(max_iterations): 
        print("\n" + "="*25 + f" REFLECTION LOOP: ITERATION {i + 1} " + "="*25) 
 
       # --- 1. GENERATE / REFINE STAGE --- 
       # In the first iteration, it generates. In subsequent iterations, it refines. 
        if i == 0: 
            print("\n>>> STAGE 1: GENERATING initial code...") 
            # The first message is just the task prompt. 
            response = llm.invoke(message_history) 
            current_code = response.content 
        else: 
            print("\n>>> STAGE 1: REFINING code based on previous critique...") 
            # The message history now contains the task,  
            # the last code, and the last critique. 
            # We instruct the model to apply the critiques. 
            message_history.append(HumanMessage(content="Please refine the code using the critiques provided.")) 
            response = llm.invoke(message_history) 
            current_code = response.content 
 
        print("\n--- Generated Code (v" + str(i + 1) + ") ---\n" + current_code) 
        message_history.append(response) # Add the generated code to history 
        print("\n>>> STAGE 2: REFLECTING on the generated code...") 

       # Create a specific prompt for the reflector agent. 
       # This asks the model to act as a senior code reviewer. 
        reflector_prompt = [ 
            SystemMessage(content=""" 
                You are a senior software engineer and an expert  
                in Python. 
                Your role is to perform a meticulous code review. 
                Critically evaluate the provided Python code based  
                on the original task requirements. 
                Look for bugs, style issues, missing edge cases,  
                and areas for improvement. 
                If the code is perfect and meets all requirements, 
                respond with the single phrase 'CODE_IS_PERFECT'. 
                Otherwise, provide a bulleted list of your critiques. 
            """), 
           HumanMessage(content=f"Original Task:\n{task_prompt}\n\nCode to Review:\n{current_code}") 
       ] 

        critique_response = llm.invoke(reflector_prompt) 
        critique = critique_response.content

        if "CODE_IS_PERFECT" in critique: 
            print("\n--- Critique ---\nNo further critiques found. The code is satisfactory.") 
            break 
 
        print("\n--- Critique ---\n" + critique) 
        # Add the critique to the history for the next refinement loop. 
        message_history.append(HumanMessage(content=f"Critique of the previous code:\n{critique}")) 
 
    print("\n" + "="*30 + " FINAL RESULT " + "="*30) 
    print("\nFinal refined code after the reflection process:\n") 
    print(current_code)

In [4]:
run_reflection_loop()


========================= REFLECTION LOOP: ITERATION 1 =========================

>>> STAGE 1: GENERATING initial code...


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError("HTTPConnection(host='localhost', port=11434): Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it"))